In [2]:
"""
Applied Data Science – Vorlesung 1

Dieses Skript implementiert GENAU die in Teil A definierten Konzepte:
- Listen/Vektoren, Mittelwert, Varianz (mit n im Nenner), Kovarianz, Korrelation, Standardisierung
- 1D-Regression: a = ȳ - b x̄, b = Cov(x, y) / Var(x)
- MSE als Zielfunktion
- Gradient Descent in 1D (nur mit den Ableitungsregeln aus A3)
- Mehrere Merkmale mit Designmatrix + eingebautem Least-Squares-Solver (ohne LA-Details)
- Train/Test-Aufteilung und Test-MSE
Hinweis: Nur NumPy wird genutzt.
"""

import numpy as np

# ------------------------------------------------------------
# 0) Statistik-Helfer (konsistent zu A1: Mittelwert, Varianz(n), Kovarianz(n), Korrelation)
# ------------------------------------------------------------
def mean(v):
    return np.sum(v) / v.shape[0]

def var_n(v):  # Varianz mit n im Nenner (wie in A1/A7)
    m = mean(v)
    return np.sum((v - m)**2) / v.shape[0]

def std_n(v):
    return np.sqrt(var_n(v))

def cov_n(x, y):  # Kovarianz mit n im Nenner (wie in A1/A7)
    mx, my = mean(x), mean(y)
    return np.sum((x - mx) * (y - my)) / x.shape[0]

def corr_xy(x, y):
    sx, sy = std_n(x), std_n(y)
    if sx == 0 or sy == 0: 
        return 0.0
    return cov_n(x, y) / (sx * sy)

def standardize(x):
    mu, sd = mean(x), std_n(x)
    z = (x - mu) / (sd + 1e-12)
    return z, mu, sd

def mse(y_true, y_pred):
    r = y_true - y_pred
    return np.mean(r**2)

# ------------------------------------------------------------
# 1) 1D-Regression genau wie in A7
# ------------------------------------------------------------
x = np.array([30, 40, 50, 60, 70], dtype=float)
y = np.array([ 2,  3,  5,  4,  6], dtype=float)  # y in 100 €

xb, yb = mean(x), mean(y)
b = cov_n(x, y) / var_n(x)   # b = Cov/Var (mit n)
a = yb - b * xb              # a = ȳ - b x̄
y_hat = a + b * x

print("1D-Regression (Formeln): a = %.3f, b = %.3f, MSE = %.4f" % (a, b, mse(y, y_hat)))

# ------------------------------------------------------------
# 2) Gradient Descent in 1D – genau gemäß A3 (Ableitungen)
#    MSE(a,b) = (1/n) * sum (y_i - (a + b x_i))^2
#    dMSE/da = (-2/n) * sum (y_i - (a + b x_i))
#    dMSE/db = (-2/n) * sum (x_i * (y_i - (a + b x_i)))
# ------------------------------------------------------------
def grad_mse_1d(a, b, x, y):
    n = x.shape[0]
    y_pred = a + b * x
    da = (-2.0 / n) * np.sum(y - y_pred)
    db = (-2.0 / n) * np.sum(x * (y - y_pred))
    return da, db

def fit_gd_1d(x, y, a0=0.0, b0=0.0, lr=1e-3, steps=20000):
    a, b = a0, b0
    for _ in range(steps):
        da, db = grad_mse_1d(a, b, x, y)
        a -= lr * da
        b -= lr * db
    return a, b

a_gd, b_gd = fit_gd_1d(x, y, lr=1e-3, steps=20000)
print("1D-GD: a = %.3f, b = %.3f, MSE = %.4f" % (a_gd, b_gd, mse(y, a_gd + b_gd * x)))

# ------------------------------------------------------------
# 3) Mehrere Merkmale: Designmatrix + Least Squares (ohne Pseudoinverse erklären)
#    Wir erzeugen synthetische Daten mit 2 Merkmalen und nutzen einen eingebauten Least-Squares-Solver.
# ------------------------------------------------------------
rng = np.random.default_rng(0)
n, d = 120, 2
X_feats = rng.normal(size=(n, d))          # zwei Merkmale/Spalten
w_true = np.array([2.0, -3.0])
bias_true = 1.5
noise = rng.normal(0, 0.8, size=n)
y_multi = bias_true + X_feats @ w_true + noise

# Designmatrix: linke Spalte = 1 für den Achsenabschnitt 'a'
X_design = np.c_[np.ones(n), X_feats]      # Form (n, d+1)
# Eingebauter Least-Squares-Solver: minimiert direkt die MSE
w_hat, *_ = np.linalg.lstsq(X_design, y_multi, rcond=None)  # w_hat = [a, w1, w2]
mse_multi = mse(y_multi, X_design @ w_hat)
print("Mehrdimensional (LS-Solver): w_hat =", np.round(w_hat, 3), "| MSE =", round(mse_multi, 3))

# ------------------------------------------------------------
# 4) Standardisierung & Train/Test – Definitionen aus A6 umgesetzt
# ------------------------------------------------------------
# Train/Test-Split: zufällige Permutation, 75%/25%
perm = rng.permutation(n)
n_train = int(0.75 * n)
idx_tr, idx_te = perm[:n_train], perm[n_train:]

X_tr, X_te = X_feats[idx_tr], X_feats[idx_te]
y_tr, y_te = y_multi[idx_tr], y_multi[idx_te]

# Standardisierung NUR auf Train fitten, dann auf Test anwenden
mu = X_tr.mean(axis=0)
sd = X_tr.std(axis=0) + 1e-12
X_tr_std = (X_tr - mu) / sd
X_te_std = (X_te - mu) / sd

# Designmatrizen wieder mit 1-Spalte:
Xtr = np.c_[np.ones(X_tr_std.shape[0]), X_tr_std]
Xte = np.c_[np.ones(X_te_std.shape[0]), X_te_std]

# Least Squares auf Train, MSE auf Train und Test berichten
w_tr, *_ = np.linalg.lstsq(Xtr, y_tr, rcond=None)
mse_tr = mse(y_tr, Xtr @ w_tr)
mse_te = mse(y_te, Xte @ w_tr)
print("Standardisiert + Train/Test: MSE_train =", round(mse_tr,3), "| MSE_test =", round(mse_te,3))

# ------------------------------------------------------------
# 5) Korrelation der Merkmale (definiert in A1)
# ------------------------------------------------------------
def corr_matrix_cols(M):
    # M: (n, d) – berechnet paarweise Korrelationen der Spalten gemäß Corr = Cov/(Std*Std)
    d = M.shape[1]
    R = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            R[i, j] = corr_xy(M[:, i], M[:, j])
    return R

R = corr_matrix_cols(X_feats)
print("Korrelationsmatrix der Features:\n", np.round(R, 3))


1D-Regression (Formeln): a = -0.500, b = 0.090, MSE = 0.3800
1D-GD: a = nan, b = nan, MSE = nan
Mehrdimensional (LS-Solver): w_hat = [ 1.469  1.992 -3.12 ] | MSE = 0.632
Standardisiert + Train/Test: MSE_train = 0.554 | MSE_test = 0.915
Korrelationsmatrix der Features:
 [[1.    0.147]
 [0.147 1.   ]]


/var/folders/nw/k_k0_cbj7vl_npdmryvhl53c0000gn/T/ipykernel_37339/1750272478.py:71: RuntimeWarning: overflow encountered in multiply
  db = (-2.0 / n) * np.sum(x * (y - y_pred))
/var/folders/nw/k_k0_cbj7vl_npdmryvhl53c0000gn/T/ipykernel_37339/1750272478.py:79: RuntimeWarning: invalid value encountered in scalar subtract
  b -= lr * db
